# Import

In [74]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import  LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from datetime import datetime,timedelta

import re
import optuna
from optuna.integration import XGBoostPruningCallback

# Data load

---
`train` : train 데이터 셋  
`test` : test 데이터 셋  
`submission` : 제출 파일  
`interest_rate` : 금리 데이터셋  
`sub_data` : 전세 가격 데이터셋

---

In [75]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')
interest_rate = pd.read_csv('interest_rate.csv')
sub_data = pd.read_csv('sub_data (1).csv')

In [76]:
train.head(2)

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion
0,TRAIN_0000,서울특별시 강남구 대치동,503,개포우성1,158.54,201401,7,174000,13,1983
1,TRAIN_0001,서울특별시 강남구 대치동,503,개포우성1,127.61,201401,9,157500,6,1983


# Preprocessing

In [77]:
train['train_test_sub'] = 'train'
test['train_test_sub'] = 'test'
sub_data['train_test_sub'] = 'sub'
all_data = pd.concat([train, test,sub_data])
all_data = all_data.reset_index(drop=True)

## Date

In [78]:
def make_date(row):
    #print(type(row['월일']))
    month_day = row['월일'].replace('월 ', '-')
    month_day = month_day.replace('일', '')
    date = str(row['연도'])+ '-' + month_day
    return date

interest_rate['날짜'] = interest_rate.apply(lambda x: make_date(x), axis=1)
interest_rate['날짜'] = pd.to_datetime(interest_rate['날짜'])

In [79]:
def preprocess_date(x):
    if type(x) == int:
        if x < 10:
            return '0'+str(x)
        else:
            return str(x)
    else:
        return x

all_data['transaction_day'] = all_data['transaction_day'].apply(preprocess_date)
all_data['transaction_date'] = all_data['transaction_year_month'].astype(int).astype(str) + all_data['transaction_day'].astype(str)
all_data['transaction_date'] = pd.to_datetime(all_data['transaction_date'])
all_data['transaction_year'] = all_data['transaction_date'].dt.year
all_data['transaction_month'] = all_data['transaction_date'].dt.month

In [80]:
all_data[(all_data['transaction_year'] == 2022)&(all_data['apt_name']=='개포우성1')].head()

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,train_test_sub,transaction_date,transaction_year,transaction_month
5869,TRAIN_5869,서울특별시 강남구 대치동,503,개포우성1,158.54,202203,19,510000.0,12,1983,train,2022-03-19,2022,3
5870,TRAIN_5870,서울특별시 강남구 대치동,503,개포우성1,127.61,202203,24,375000.0,1,1983,train,2022-03-24,2022,3
5871,TRAIN_5871,서울특별시 강남구 대치동,503,개포우성1,127.61,202203,30,409000.0,11,1983,train,2022-03-30,2022,3
5872,TRAIN_5872,서울특별시 강남구 대치동,503,개포우성1,84.81,202205,20,323000.0,14,1983,train,2022-05-20,2022,5
5873,TRAIN_5873,서울특별시 강남구 대치동,503,개포우성1,84.81,202207,10,310000.0,14,1983,train,2022-07-10,2022,7


In [82]:
interest_rate['year'] = interest_rate['날짜'].dt.year
interest_rate['month'] = interest_rate['날짜'].dt.month
interest_rate['day'] = interest_rate['날짜'].dt.day
interest_rate = interest_rate[['날짜','year','month','day','금리']]
interest_rate = interest_rate[interest_rate['year']>=2013]
interest_rate

,날짜,year,month,day,금리
0,2023-01-13,2023,1,13,3.50
1,2022-11-24,2022,11,24,3.25
2,2022-10-12,2022,10,12,3.00
3,2022-08-25,2022,8,25,2.50
4,2022-07-13,2022,7,13,2.25
5,2022-05-26,2022,5,26,1.75
6,2022-04-14,2022,4,14,1.50
7,2022-01-14,2022,1,14,1.25
8,2021-11-25,2021,11,25,1.00
9,2021-08-26,2021,8,26,0.75


In [114]:
for idx, row in tqdm(all_data.iterrows(), total = all_data.shape[0]):
    date = row['transaction_date']
    rate = interest_rate[interest_rate['날짜'] <= date].iloc[0]['금리']
    all_data.loc[idx, 'interest_rate'] = rate


100%|██████████████████████████████████████████████████████████████████| 23984/23984 [00:04<00:00, 5148.53it/s]


# Feature Engineering

In [115]:
# 전용면적 40 이하 데이터 개수 확인
print(sum(all_data['exclusive_use_area']<40))
print(sum(all_data['exclusive_use_area']<30))

856
218


In [116]:
tmp_train = all_data[all_data['train_test_sub']=='train']
tmp_test = all_data[all_data['train_test_sub']=='test']
tmp_sub = all_data[all_data['train_test_sub']=='sub']

# test셋에만 있는 apt_name 이 있는지 확인
print(set(tmp_test['apt_name'].unique()) - set(tmp_train['apt_name'].unique()))
print(set(tmp_test['apt_name'].unique()) - set(tmp_sub['apt_name'].unique()))

{'(977-)'}
set()


## make_area_bucket

In [117]:
def make_area_bucket(area):
    if area <= 40:
        return 0
    elif area < 60: # 59타입
        return 1
    elif area < 85: # 84타입
        return 2
    else:
        return 3
    
# 아파트 면적 전처리
all_data['bucket_area'] = all_data['exclusive_use_area'].apply(make_area_bucket)
all_data['bucket_area'].value_counts()

bucket_area
2    12771
3     8635
1     1722
0      856
Name: count, dtype: int64

## make_new_area
- 10단위로 새로운 면적 feature를 생성

In [118]:
def make_new_area(area):
    return (area//10)*10
    
# 아파트 면적 전처리
all_data['new_area'] = all_data['exclusive_use_area'].apply(make_new_area)
all_data['new_area'] = all_data['new_area'].astype(int)
all_data['new_area'].value_counts()

new_area
80     8153
70     4772
120    2485
110    1783
50     1641
90     1404
140     678
160     664
30      638
130     420
100     412
150     353
20      218
60      102
190      84
40       81
170      48
180      35
200      13
Name: count, dtype: int64

## ver2_all_data  
test셋에 있는 apt_name 데이터만 사용하기위해 새로운 df 선언  
전세 데이터는 제외


In [119]:
#ver2_all_data = all_data[all_data['train_test_sub']!='sub'].copy()
ver2_all_data = all_data.copy()

test_apt_name = sorted(list(tmp_test['apt_name'].unique()))
ver2_all_data = ver2_all_data[ver2_all_data['apt_name'].isin(test_apt_name)]
print('데이터 개수 : ',len(all_data),'>',len(ver2_all_data))
ver2_all_data.tail()

데이터 개수 :  23984 > 21864


,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,train_test_sub,transaction_date,transaction_year,transaction_month,bucket_area,new_area,interest_rate
23974,NaN,서울특별시 강남구 대치동,992,현대1,84.27,202303,25,50000.0,14,1990,sub,2023-03-25,2023,3,2,80,3.5
23975,NaN,서울특별시 강남구 대치동,992,현대1,84.89,202305,23,52500.0,6,1990,sub,2023-05-23,2023,5,2,80,3.5
23976,NaN,서울특별시 강남구 대치동,992,현대1,84.89,202305,23,52500.0,6,1990,sub,2023-05-23,2023,5,2,80,3.5
23977,NaN,서울특별시 강남구 대치동,992,현대1,84.27,202305,27,45000.0,14,1990,sub,2023-05-27,2023,5,2,80,3.5
23978,NaN,서울특별시 강남구 대치동,992,현대1,84.27,202306,17,40000.0,10,1990,sub,2023-06-17,2023,6,2,80,3.5


In [120]:
test_apt_name

['(977-)',
 '개포우성1',
 '개포우성2',
 '대우아이빌멤버스(891-26)',
 '대우아이빌명문가(891-23)',
 '대치SKVIEW',
 '대치삼성',
 '대치아이파크',
 '대치우성아파트1동,2동,3동,5동,6동,7동',
 '대치현대',
 '대치효성',
 '동부센트레빌',
 '래미안대치팰리스',
 '래미안대치하이스턴',
 '선경1차(1동-7동)',
 '선경2차(8동-12동)',
 '선릉역대우아이빌(890-49)',
 '쌍용대치2',
 '쌍용대치아파트1동,2동,3동,5동,6동',
 '은마',
 '테헤란로대우아이빌(891-6)',
 '포스코더샵',
 '풍림아이원2차202동',
 '한보미도맨션1',
 '한보미도맨션2',
 '현대1']

In [121]:
def change_apt_name(all_data,apt_lst,result_name):
    print(all_data[all_data['apt_name'].isin(apt_lst)]['apt_name'].value_counts())
    print('sum :', len(all_data[all_data['apt_name'].isin(apt_lst)]['apt_name']))
    all_data['apt_name'] = all_data['apt_name'].map(lambda x: result_name if x in apt_lst else x)
    print('check :',len(all_data[all_data['apt_name']==result_name]))
    print('---'*30)
    return all_data

ver2_all_data = change_apt_name(ver2_all_data,['선경1차(1동-7동)','선경2차(8동-12동)'],'선경')
ver2_all_data = change_apt_name(ver2_all_data,['한보미도맨션1','한보미도맨션2'],'한보미도맨션')

apt_name
선경1차(1동-7동)     791
선경2차(8동-12동)    513
Name: count, dtype: int64
sum : 1304
check : 1304
------------------------------------------------------------------------------------------
apt_name
한보미도맨션2    1228
한보미도맨션1    1117
Name: count, dtype: int64
sum : 2345
check : 2345
------------------------------------------------------------------------------------------


In [122]:
# exclusive_use_area 의 데이터 타입 변경
ver2_all_data['exclusive_use_area'] = ver2_all_data['exclusive_use_area'].astype(int)

# 다시 그룹화 개수 확인
print(len(ver2_all_data.groupby(['apt_name','exclusive_use_area'])['transaction_real_price'].mean().index))

101


### new apt_id

- 'apt_name'과 'exclusive_area' 별로 그룹화 하려고 했으나 그룹이 너무 많아서 아파트 이름으로만 apt_id 를 만들기로함.

In [123]:
label_encoder = LabelEncoder()
ver2_all_data['apt_id'] = label_encoder.fit_transform(ver2_all_data['apt_name'])
print(min(ver2_all_data['apt_id']),max(ver2_all_data['apt_id']))
print(label_encoder.classes_)

0 23
['(977-)' '개포우성1' '개포우성2' '대우아이빌멤버스(891-26)' '대우아이빌명문가(891-23)' '대치SKVIEW'
 '대치삼성' '대치아이파크' '대치우성아파트1동,2동,3동,5동,6동,7동' '대치현대' '대치효성' '동부센트레빌'
 '래미안대치팰리스' '래미안대치하이스턴' '선경' '선릉역대우아이빌(890-49)' '쌍용대치2'
 '쌍용대치아파트1동,2동,3동,5동,6동' '은마' '테헤란로대우아이빌(891-6)' '포스코더샵' '풍림아이원2차202동'
 '한보미도맨션' '현대1']


## ver3_all_data
ver2는 apt_name 으로만 구분지어 apt_id 를 만든 반면  
ver3에서는 apt_name과 더불어 면적에 관한 feature도 함께 그룹화 하고자 함.

In [124]:
ver3_all_data = all_data.copy()

test_apt_name = sorted(list(tmp_test['apt_name'].unique()))
ver3_all_data = ver3_all_data[ver3_all_data['apt_name'].isin(test_apt_name)]
ver3_all_data.tail()

,id,sigungu,jibun,apt_name,exclusive_use_area,transaction_year_month,transaction_day,transaction_real_price,floor,year_of_completion,train_test_sub,transaction_date,transaction_year,transaction_month,bucket_area,new_area,interest_rate
23974,NaN,서울특별시 강남구 대치동,992,현대1,84.27,202303,25,50000.0,14,1990,sub,2023-03-25,2023,3,2,80,3.5
23975,NaN,서울특별시 강남구 대치동,992,현대1,84.89,202305,23,52500.0,6,1990,sub,2023-05-23,2023,5,2,80,3.5
23976,NaN,서울특별시 강남구 대치동,992,현대1,84.89,202305,23,52500.0,6,1990,sub,2023-05-23,2023,5,2,80,3.5
23977,NaN,서울특별시 강남구 대치동,992,현대1,84.27,202305,27,45000.0,14,1990,sub,2023-05-27,2023,5,2,80,3.5
23978,NaN,서울특별시 강남구 대치동,992,현대1,84.27,202306,17,40000.0,10,1990,sub,2023-06-17,2023,6,2,80,3.5


In [125]:
ver3_all_data['exclusive_use_area'] = ver3_all_data['exclusive_use_area'].astype(int)
for tmp in ver3_all_data['apt_name'].unique():
    print(tmp,' : ',sorted(ver3_all_data[ver3_all_data['apt_name']==tmp]['new_area'].unique()))

개포우성1  :  [80, 120, 150, 180]
개포우성2  :  [80, 120, 150]
대우아이빌멤버스(891-26)  :  [20, 30, 40, 50]
대우아이빌명문가(891-23)  :  [30, 40, 50]
대치삼성  :  [50, 80, 90, 100]
대치아이파크  :  [50, 80, 110, 140]
대치우성아파트1동,2동,3동,5동,6동,7동  :  [80, 110]
대치현대  :  [50, 80, 100, 110]
대치효성  :  [50, 70, 80]
동부센트레빌  :  [120, 140, 160]
래미안대치하이스턴  :  [110]
선경1차(1동-7동)  :  [80, 110, 130, 160]
선경2차(8동-12동)  :  [80, 120, 160]
선릉역대우아이빌(890-49)  :  [20, 40]
쌍용대치2  :  [80, 120]
쌍용대치아파트1동,2동,3동,5동,6동  :  [80, 120, 140]
은마  :  [70, 80]
테헤란로대우아이빌(891-6)  :  [20, 30, 50, 60]
포스코더샵  :  [130, 160]
한보미도맨션1  :  [80, 120, 160, 190]
한보미도맨션2  :  [80, 110, 120, 150, 190]
현대1  :  [80]
래미안대치팰리스  :  [50, 80, 90, 110, 150]
풍림아이원2차202동  :  [110, 130, 150]
대치SKVIEW  :  [50, 80, 90, 110, 120]
(977-)  :  [50, 70, 80]


### new apt_id  
- apt_name 과 new_area 로 그룹화하여 새로운 apt_id feature 생성

In [126]:
ver3_all_data['apt_id'] = ver3_all_data.groupby(['apt_name','new_area']).ngroup()
print(min(ver3_all_data['apt_id']),max(ver3_all_data['apt_id']))

set(ver3_all_data[ver3_all_data['train_test_sub']=='test']['apt_id'].unique()) -\
set(ver3_all_data[ver3_all_data['train_test_sub']!='test']['apt_id'].unique())

0 82


set()

## get_recent_price

In [127]:
def get_recent_price(idx, all_data,train=1):
    '''
    idx : index
    all_data : 데이터 셋
    train : 1일 경우 매매가격 return, 0일 경우 전세 가격 return 
    
    return : 현재 계약일자보다 과거에 이루어진 같은 apt_id의 계약 중에 
             가장 최근의 매매/전세 계약금액을 return 
             
             만약 현재 계약일자보다 과거에 이루어진 같은 apt_id의 계약이 없다면 
             현재 계약연도와 같고 같은 bucket_area 값을 가진 데이터들의 
             매매/전세 계약금액을 평균을 return 
             
              ㄴ 이 과정에서 미래의 데이터를 사용할 여지가 있음. 
                ex) 현재 계약연도가 2023년이지만 현재 계약일자보다 과거에 이루어진 같은 apt_id의 계약이 없는 경우
                    현재 계약일자보다 더 미래의 계약일자의 계약 금액이 평균값에 포함될 수 있음. (전세 계약금액을 구하는 경우)
                
                하지만 그런 경우 없는 것 확인(check_list)
    
    '''
    temp_df = all_data.loc[:idx] # 현재 시간까지 전체 데이터
    check=False
    

    if train:
        temp_df = temp_df[
            (temp_df['train_test_sub'] == 'train') &
            (temp_df['transaction_date'] < row['transaction_date']) &
            (temp_df['apt_id'] == row['apt_id']) 
        ]
        
        if len(temp_df) == 0:
            check=True
            temp_df = all_data[
                (all_data['train_test_sub'] == 'train') &
                (all_data['transaction_year'] == row['transaction_year']) &
                (all_data['bucket_area'] == row['bucket_area']) 
            ]
    else:
        temp_df = temp_df[
            (temp_df['train_test_sub'] == 'sub') &
            (temp_df['transaction_date'] < row['transaction_date']) &
            (temp_df['apt_id'] == row['apt_id']) 
        ]
        
        if len(temp_df) == 0:
            check=True
            temp_df = all_data[
                (all_data['train_test_sub'] == 'sub') &
                (all_data['transaction_year'] == row['transaction_year']) &
                (all_data['bucket_area'] == row['bucket_area']) 
            ]
            
    if check:
        # 현재 계약일 이전 같은 apt_id의 계약이 없는 경우의 계약연도 
        check_list.append(all_data.loc[idx,'transaction_year'])
        recent_price = temp_df['transaction_real_price'].mean()
    else:
        recent_price = temp_df.iloc[-1]['transaction_real_price']
        
    return recent_price

In [128]:
# ver2_all_data
ver2_all_data = ver2_all_data.sort_values(by=['transaction_date','id']).reset_index(drop=True)
global check_list
check_list = []
for idx, row in tqdm(ver2_all_data.iterrows(), total = ver2_all_data.shape[0]):
    if row['train_test_sub'] == 'sub':  
        continue
    ver2_all_data.loc[idx, 'recent_rent_price'] = get_recent_price(idx, ver2_all_data,0)

100%|██████████████████████████████████████████████████████████████████| 21864/21864 [00:04<00:00, 4921.72it/s]


In [129]:
print(check_list)

[2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014]


In [130]:
# ver3_all_data
ver3_all_data = ver3_all_data.sort_values(by=['transaction_date','id']).reset_index(drop=True)
check_list = []
global check_list
for idx, row in tqdm(ver3_all_data.iterrows(), total = ver3_all_data.shape[0]):
    if row['train_test_sub'] == 'sub':  
        continue
    ver3_all_data.loc[idx, 'recent_rent_price'] = get_recent_price(idx, ver3_all_data,0)
print(check_list)

100%|██████████████████████████████████████████████████████████████████| 21864/21864 [00:04<00:00, 5259.94it/s]

[2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2016]


In [131]:
ver2_all_data.columns

Index(['id', 'sigungu', 'jibun', 'apt_name', 'exclusive_use_area',
       'transaction_year_month', 'transaction_day', 'transaction_real_price',
       'floor', 'year_of_completion', 'train_test_sub', 'transaction_date',
       'transaction_year', 'transaction_month', 'bucket_area', 'new_area',
       'interest_rate', 'apt_id', 'recent_rent_price'],
      dtype='object')

최근 전세 데이터를 잘 불러오는지 확인

In [133]:
ver2_all_data[ver2_all_data['apt_id']==18][13:16][['apt_name','transaction_real_price','train_test_sub','transaction_date','recent_rent_price']]

,apt_name,transaction_real_price,train_test_sub,transaction_date,recent_rent_price
48,은마,37000.0,sub,2014-01-07,NaN
49,은마,35000.0,sub,2014-01-07,NaN
52,은마,77500.0,train,2014-01-08,35000.0


In [134]:
ver2_all_data[ver2_all_data['apt_id']==19][380:390][['apt_name','transaction_real_price','train_test_sub','transaction_date','recent_rent_price']]

,apt_name,transaction_real_price,train_test_sub,transaction_date,recent_rent_price
21107,테헤란로대우아이빌(891-6),32000.0,sub,2023-02-27,NaN
21164,테헤란로대우아이빌(891-6),NaN,test,2023-03-10,32000.0
21277,테헤란로대우아이빌(891-6),32000.0,sub,2023-03-24,NaN
21294,테헤란로대우아이빌(891-6),28000.0,sub,2023-03-25,NaN
21324,테헤란로대우아이빌(891-6),30000.0,sub,2023-03-30,NaN
21402,테헤란로대우아이빌(891-6),30950.0,sub,2023-04-12,NaN
21443,테헤란로대우아이빌(891-6),NaN,test,2023-04-20,30950.0
21582,테헤란로대우아이빌(891-6),28000.0,sub,2023-05-12,NaN
21613,테헤란로대우아이빌(891-6),NaN,test,2023-05-18,28000.0
21660,테헤란로대우아이빌(891-6),50000.0,sub,2023-05-23,NaN


# Modeling

## train_valid_split

In [151]:
col = ['exclusive_use_area','floor','transaction_year', 'transaction_month', 
       'interest_rate','bucket_area', 'apt_id', 'recent_rent_price']

In [152]:
train_x = ver2_all_data[
    (ver2_all_data['train_test_sub']=='train') &
    (ver2_all_data['transaction_year'] < 2022)
][col]

train_y = ver2_all_data[
    (ver2_all_data['train_test_sub']=='train') &
    (ver2_all_data['transaction_year'] < 2022)
]['transaction_real_price']

val_x = ver2_all_data[
    (ver2_all_data['train_test_sub']=='train') &
    (ver2_all_data['transaction_year'] == 2022)
][col]

val_y = ver2_all_data[
    (ver2_all_data['train_test_sub']=='train') &
    (ver2_all_data['transaction_year'] == 2022)
]['transaction_real_price']

## train and validation

In [153]:
model_dt = DecisionTreeRegressor(random_state=1)
model_rf = RandomForestRegressor(random_state=1)
model_gb = GradientBoostingRegressor(random_state=1)
model_xgb = xgb.XGBRegressor(random_state=1)
model_lgb = lgb.LGBMRegressor(random_state=1)
model_cb = CatBoostRegressor(cat_features=['apt_id','bucket_area'],random_state=1)


In [154]:
model_dict = {'dt': model_dt, 'rf': model_rf, 'gb': model_gb,'xgb': model_xgb, 'lgb': model_lgb, 'cb': model_cb}

In [155]:
for name in model_dict.keys():
        print('model: {}'.format(name))
        model = model_dict[name]
        if name == 'cb':
            model.fit(train_x, train_y,verbose=False)
        else:
            model.fit(train_x, train_y)

model: dt
model: rf
model: gb
model: xgb
model: lgb
model: cb


In [156]:
for name in model_dict.keys():
        #print('model: {}'.format(name))
        model = model_dict[name]
        pred = model.predict(val_x)
        mae = mean_absolute_error(val_y, pred)
        print(f'{name} : {mae}')

dt : 44216.49541284404
rf : 41459.47422018349
gb : 39849.80337330492
xgb : 30108.042162413993
lgb : 36447.42919590957
cb : 30064.24174472108


catboost의 성능이 가장 좋은 것을 확인

## tuning

In [157]:
def cb_objective(trial):
    # 하이퍼파라미터 탐색 대상
    n_estimators = trial.suggest_int('n_estimators', 10, 1000, step=100)

    model = CatBoostRegressor(n_estimators=n_estimators,
                              cat_features=['apt_id','bucket_area'],
                              random_seed = 1)

    model.fit(train_x, train_y,verbose=False)

    # 검증 데이터로 평가
    y_pred = model.predict(val_x)
    mae = mean_absolute_error(val_y, y_pred)
    return mae

# Optuna를 사용하여 하이퍼파라미터 탐색
optuna.logging.set_verbosity(optuna.logging.ERROR)
study = optuna.create_study(direction='minimize')# 목표는 최소화
study.set_user_attr('seed', 42)
study.optimize(cb_objective, n_trials=50,show_progress_bar=True)  # 50회 반복하여 탐색

# 최적의 하이퍼파라미터 값 출력
best_params_cb = study.best_params
print("Best Params:", best_params_cb)
print('values: ',study.best_trial.value)

  0%|          | 0/50 [00:00<?, ?it/s]

Best Params: {'n_estimators': 310}
values:  28794.043427175126


# submit
  
 - Optuna를 사용해서 튜닝후 제출했음에도 점수가 낮아지지 않는 것을 보고  
     validaiton set을 신뢰할 수 없다고 판단함.  
     
 - 최종 제출은 ver2 데이터를 사용한 Catboost 모델과 ver3 데이터를 사용한 Catboost 모델을 앙상블하여
     최종 에측을 하였음.

In [158]:
col = ['exclusive_use_area','floor','transaction_year', 'transaction_month', 
       'interest_rate','bucket_area', 'apt_id', 'recent_rent_price']

model_ver2 = CatBoostRegressor(n_estimators=500,cat_features=['apt_id','bucket_area'],random_seed=25)

train_x = ver2_all_data[ver2_all_data['train_test_sub']=='train'].sort_values(by='id')[col]
train_y = ver2_all_data[ver2_all_data['train_test_sub']=='train'].sort_values(by='id')['transaction_real_price']
test_x = ver2_all_data[ver2_all_data['train_test_sub']=='test'].sort_values(by='id')[col]

model_ver2.fit(train_x,train_y,verbose=False)
pred_ver2 = model_ver2.predict(test_x)

In [159]:
col = ['exclusive_use_area','floor','transaction_year', 'transaction_month', 
       'interest_rate', 'apt_name', 'recent_rent_price','new_area']

model_ver3 = CatBoostRegressor(cat_features=['new_area','apt_name'],random_seed=42)

train_x = ver3_all_data[ver3_all_data['train_test_sub']=='train'].sort_values(by='id')[col]
train_y = ver3_all_data[ver3_all_data['train_test_sub']=='train'].sort_values(by='id')['transaction_real_price']
test_x = ver3_all_data[ver3_all_data['train_test_sub']=='test'].sort_values(by='id')[col]

model_ver3.fit(train_x,train_y,verbose=False)
pred_ver3 = model_ver3.predict(test_x)

In [160]:
pred = (pred_ver2 + pred_ver3)/2

In [149]:
submission['transaction_real_price'] = pred
submission.to_csv('final_submission.csv', index=False)